In [56]:
import numpy as np
from scipy.stats import norm

## Problem 1

![Problem1](https://github.com/Sally-Yitong/FINM-32000-Numerical-Methods/blob/main/HW6/HW6%20T1%20handwriting.jpg?raw=true)

In [57]:
class MultiGBM:

    def __init__(self,S0,r,correlations,sigma):
        self.S0 = S0
        self.r = r
        self.correlations = correlations
        self.sigma = sigma

In [58]:
class GBMdynamics:

    def __init__(self, S, r, rGrow, sigma=None):
        self.S = S # S0
        self.r = r # interest rate
        self.rGrow = rGrow # R_grow
        self.sigma = sigma # instantenous vol

    def update_sigma(self, sigma):
        self.sigma = sigma
        return self

In [59]:
hw6p1dynamics = MultiGBM(S0=np.array([100,110]),r=0.05,
                         correlations = np.array([[1,0.8],[0.8,1]]),
                         sigma = np.diag([0.3, 0.2]))

In [60]:
class CallOnBasket:

    def __init__(self,K,T,weights):
        self.K = K
        self.T = T
        self.weights = weights

In [61]:
class CallOption:

    def __init__(self, K, T, price=None):
        self.K = K
        self.T = T
        self.price = price

In [62]:
hw6p1contract=CallOnBasket(K=110,T=1.0,weights = np.array([1/2, 1/2]))

In [68]:
class MCengine:

    def __init__(self, M, antithetic, control, seed):
        self.M = M                                  # How many simulations
        self.antithetic = antithetic
        self.control = control
        self.rng = np.random.default_rng(seed=seed) # Seeding the random number generator with a specified number helps make the calculations reproducible
    
    def BSpriceCall(self, dynamics, contract):
        # ignores contract.price if given, because this function calculates price based on the dynamics

        F = dynamics.S*np.exp(dynamics.rGrow*contract.T)
        std = dynamics.sigma*np.sqrt(contract.T)
        d1 = np.log(F/contract.K)/std+std/2
        d2 = d1-std
        call_price = np.exp(-dynamics.r*contract.T)*(F*norm.cdf(d1)-contract.K*norm.cdf(d2))
        return call_price
    
    def price_callonbasket_multiGBM(self,contract,dynamics):

        # You complete the coding of this function.
        # self.rng.multivariate_normal may be useful.
        # See documentation for numpy.random.Generator.multivariate_normal
        # as self.rng is an instance of numpy.random.Generator

        # You are not required to support the case where MC.control = MC.antithetic = True
        # (simultaneous use of control variate and antithetic)
        # But you are required to support the other 3 possible settings of MC.antithetic/MC.control
        # namely False/False, True/False, False/True.
        # (ordinary MC, antithetic without control, control without antithetic)
        
        S0, r, correlations, sigma = dynamics.S0, dynamics.r, dynamics.correlations, dynamics.sigma
        K, T, weights = contract.K, contract.T, contract.weights
    
        X0 = np.log(S0)
        mu = np.array([r,r]) - (1/2)*np.diag((sigma@sigma))
        covariances = correlations * T
        random_corr_norm = self.rng.multivariate_normal(np.array([0,0]), covariances, size=self.M)
        
        XT = X0[:, np.newaxis] + (mu*T)[:, np.newaxis] + sigma @ random_corr_norm.T
        ST = np.exp(XT)
        HT = weights @ ST
        payoff = np.maximum(HT-K, 0)
        C0 = payoff * np.exp(-r*T)

        if self.antithetic:
            negative_random_corr_norm = -random_corr_norm
            negative_XT = X0[:, np.newaxis] + (mu*T)[:, np.newaxis] + sigma @ negative_random_corr_norm.T
            negative_ST = np.exp(negative_XT)
            negative_HT = weights @ negative_ST
            negative_payoff = np.maximum(negative_HT-K, 0)
            negative_C0 = negative_payoff * np.exp(-r*T)
            antithetic_C0 = (negative_C0 + C0)/2

        if self.control:
            log_GT = weights @ XT
            GT = np.exp(log_GT)
            geometric_payoff = np.maximum(GT-K, 0)
            geometric_C0 = geometric_payoff * np.exp(-r*T)

            beta_hat = np.cov(geometric_C0, C0)[0,1]/np.cov(geometric_C0, C0)[0,0]
            G0 = (S0[0] * S0[1]) ** (1/2)
            G_Rgrow = r - (sigma[0,0]**2 - 2*correlations[0,1]*sigma[0,0]*sigma[1,1] + sigma[1,1]**2)/8
            G_sigma = np.sqrt(sigma[0,0]**2 + 2*correlations[0,1]*sigma[0,0]*sigma[1,1] + sigma[1,1]**2)/2
            G_dynamics = GBMdynamics(G0, r, G_Rgrow, G_sigma)
            G_contract = CallOption(K,T)
            Cstar = self.BSpriceCall(G_dynamics,G_contract)

        if (not self.antithetic) and (not self.control):
            call_price = np.mean(C0)
            standard_error = np.std(C0, ddof=1)
        if self.antithetic == True and (not self.control):
            call_price = np.mean(antithetic_C0)
            standard_error = np.std(antithetic_C0, ddof=1)
        if self.control == True and (not self.antithetic):
            call_price = np.mean(C0) + beta_hat * (Cstar - np.mean(geometric_C0))
            standard_error = np.std(C0 - beta_hat * geometric_C0) / np.sqrt(self.M)
        
        return(call_price, standard_error)

### (b)

In [69]:
hw6p1bMC=MCengine(M=10000,antithetic=False,control=False,seed=0)
(call_price_ordinary, std_err_ordinary) = hw6p1bMC.price_callonbasket_multiGBM(hw6p1contract,hw6p1dynamics)
print(call_price_ordinary, std_err_ordinary)

9.858103798706601 16.80048813661487


### (c)

In [70]:
hw6p1cMC=MCengine(M=10000,antithetic=True,control=False,seed=0)
(call_price_AV, std_err_AV) = hw6p1cMC.price_callonbasket_multiGBM(hw6p1contract,hw6p1dynamics)
print(call_price_AV, std_err_AV)

9.930408770857396 9.477429058267408


### (f)

In [71]:
hw6p1fMC=MCengine(M=10000,antithetic=False,control=True,seed=0)
(call_price_CV, std_err_CV) = hw6p1fMC.price_callonbasket_multiGBM(hw6p1contract,hw6p1dynamics)
print(call_price_CV, std_err_CV)

9.993510290823448 0.004473505779267341


## Problem 2

In [72]:
class GBM:

    def __init__(self,sigma,r,S0):
        self.sigma = sigma
        self.r = r
        self.S0 = S0

In [73]:
hw6p2dynamics=GBM(sigma=0.2,r=0.02,S0=100)

In [74]:
class CallOption:

    def __init__(self,K,T):
        self.K=K
        self.T=T

In [75]:
hw6p2contract=CallOption(K=150,T=1)

In [76]:
class MCimportanceEngine:

    def __init__(self, M, lamb, seed):
        self.M = M                                  # How many simulations
        self.lamb = lamb                            # drift adjustment
        self.rng = np.random.default_rng(seed=seed) # Seeding the random number generator with a specified number helps make the calculations reproducible

    def price_call_GBM(self, contract,dynamics):

        # You complete the coding of this function.
        # self.rng.normal may be useful.
        # See documentation for numpy.random.Generator.normal
        # as self.rng is an instance of numpy.random.Generator

        W1 = self.rng.normal(0, 1, self.M) 
        XT = np.log(dynamics.S0) + (dynamics.r-dynamics.sigma**2/2)*contract.T + dynamics.sigma*np.sqrt(contract.T)*(W1 + self.lamb * contract.T)
        ST = np.exp(XT)
        YT = np.exp(-dynamics.r*contract.T - self.lamb*W1*np.sqrt(contract.T) - 0.5*contract.T* self.lamb**2) * np.maximum(ST-contract.K, 0)
        
        call_price = np.mean(YT)
        standard_error = np.std(YT, ddof=1) / np.sqrt(self.M)

        return(call_price, standard_error)


### (a)

In [77]:
hw6p2aMC=MCimportanceEngine(M=100000,lamb=0,seed=0) #zero drift adjustment gives ordinary MC

(call_price_ordinary, std_err_ordinary) =  hw6p2aMC.price_call_GBM(hw6p2contract,hw6p2dynamics)
print(call_price_ordinary, std_err_ordinary)

0.25270332833609405 0.007609293292996182


### (b)
![T2](https://github.com/Sally-Yitong/FINM-32000-Numerical-Methods/blob/main/HW6/HW6%20T2.jpg?raw=true)

In [81]:
ES_T = 165
S_0 = hw6p2dynamics.S0
r = hw6p2dynamics.r
sigma = hw6p2dynamics.sigma
T = hw6p2contract.T

lamb= (np.log(ES_T/S_0)/T - r) / sigma
print(lamb)

2.4038764395624455


### (c)

In [79]:
hw6p2cMC=MCimportanceEngine(M=100000,lamb=lamb,seed=0) # Fill in the lamb parameter with the lambda that you compute in (b)
(call_price_importsamp, std_err_importsamp) =  hw6p2cMC.price_call_GBM(hw6p2contract,hw6p2dynamics)
print(call_price_importsamp, std_err_importsamp)

0.24843662621391502 0.0007734271968138013
